https://shubham-shinde.github.io/blogs/arcface/ - spoko wyjaśnienie czym jest ArcFace

In [3]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Lambda, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_addons as tfa
# !pip install focal-loss
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, Input
from sklearn.model_selection import train_test_split
import shutil
from tensorflow.keras import backend as K
from tensorflow.keras import layers, models, regularizers
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers.legacy import Adam
from focal_loss import BinaryFocalLoss


In [4]:
# glowny folder
base_dir = '/Users/milenabiernacka/Desktop/studia/DS/semestr2/PD-magisterka/Mushroom_dataset/cnn/'
# polaczenie glownego folderu i podfolderow z edible i poisonous

edible_dir = os.path.join(base_dir, 'Edible')
poisonous_dir = os.path.join(base_dir, 'Poisonous')

# Assuming base_dir is the path to the dataset directory
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size = 32
img_height = 128
img_width = 128

train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')


Found 1399 images belonging to 2 classes.
Found 300 images belonging to 2 classes.
Found 301 images belonging to 2 classes.


In [5]:
def wrapper_generator(generator):
    for batch_x, batch_y in generator:
        yield [batch_x, batch_y], batch_y

In [6]:
train_gen_wrapper = wrapper_generator(train_generator)
validation_gen_wrapper = wrapper_generator(validation_generator)
test_gen_wrapper = wrapper_generator(test_generator)

In [7]:
train_gen_wrapper

<generator object wrapper_generator at 0x310e92880>

In [8]:
class ArcFaceLossLayer(tf.keras.layers.Layer):
    def __init__(self, n_classes=2, s=30.0, m=0.50, **kwargs):
        super(ArcFaceLossLayer, self).__init__(**kwargs)
        self.n_classes = n_classes
        self.s = s  # Scale parameter
        self.m = m  # Margin parameter
        self.cos_m = tf.cos(m)
        self.sin_m = tf.sin(m)
        self.th = tf.cos(tf.constant(np.pi) - m)
        self.mm = tf.sin(tf.constant(np.pi) - m) * m

    def build(self, input_shape):
        self.W = self.add_weight(name='W',
                                 shape=(input_shape[-1], self.n_classes),
                                 initializer='glorot_uniform',
                                 trainable=True,
                                 dtype='float32')

    def call(self, inputs, labels):
        # Normalize feature vectors and weights
        x = tf.nn.l2_normalize(inputs, axis=1)
        W = tf.nn.l2_normalize(self.W, axis=0)
        # Dot product
        logits = x @ W
        # Add margin
        theta = tf.acos(tf.clip_by_value(logits, -1.0 + 1e-7, 1.0 - 1e-7))
        target_logits = tf.cos(theta + self.m)
        logits = logits * (1 - labels) + target_logits * labels
        # Rescale logits
        return logits * self.s


In [15]:
def build_model_with_arcface_loss(input_shape=(150, 150, 3), n_classes=2):
    inputs = Input(shape=input_shape)
    x = Conv2D(32, (3, 3), activation='relu')(inputs)
    x = MaxPooling2D(2, 2)(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D(2, 2)(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = MaxPooling2D(2, 2)(x)
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    # No activation on the last dense layer
    x = Dense(128)(x)  # This is the embedding layer
    # Note: ArcFace layer is typically applied during training only

    # Placeholder for labels
    labels = Input(shape=(n_classes,))
    # Instantiate ArcFace loss layer
    arcface_loss = ArcFaceLossLayer(n_classes=n_classes)(x, labels)

    model = Model(inputs=[inputs, labels], outputs=arcface_loss)
    return model


In [16]:
model = build_model_with_arcface_loss(input_shape=(150, 150, 3), n_classes=2)

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss=[tf.nn.softmax_cross_entropy_with_logits],
              metrics=['accuracy'])

# When fitting the model, ensure to pass both the images and labels as input,
# and similarly for validation. You might need to adjust your data generator
# or data loading mechanism to achieve this.


In [17]:
model.fit(
    train_gen_wrapper,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=validation_gen_wrapper,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/20
43/43 [==============================] - 11s 254ms/step - loss: 6.4426 - accuracy: 0.5022 - val_loss: 5.1759 - val_accuracy: 0.7361
Epoch 2/20
43/43 [==============================] - 11s 248ms/step - loss: 5.3858 - accuracy: 0.5260 - val_loss: 4.5934 - val_accuracy: 0.7313
Epoch 3/20
43/43 [==============================] - 11s 252ms/step - loss: 4.7664 - accuracy: 0.5523 - val_loss: 3.6961 - val_accuracy: 0.6940
Epoch 4/20
43/43 [==============================] - 11s 258ms/step - loss: 3.4335 - accuracy: 0.6042 - val_loss: 2.7667 - val_accuracy: 0.7015
Epoch 5/20
43/43 [==============================] - 12s 271ms/step - loss: 2.5074 - accuracy: 0.6964 - val_loss: 2.0815 - val_accuracy: 0.7687
Epoch 6/20
43/43 [==============================] - 11s 262ms/step - loss: 1.7464 - accuracy: 0.7740 - val_loss: 2.1203 - val_accuracy: 0.7873
Epoch 7/20
43/43 [==============================] - 11s 265ms/step - loss: 1.4317 - accuracy: 0.8149 - val_loss: 1.4718 - val_accuracy: 0.8507

In [18]:
test_loss, test_accuracy = model.evaluate(test_gen_wrapper, steps=test_generator.samples // test_generator.batch_size)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

9/9 [==============================] - 1s 75ms/step - loss: 1.4404 - accuracy: 0.8819
Test Loss: 1.440354585647583, Test Accuracy: 0.8819444179534912


# Pretrenowane sieci

### MobileNetV2

In [13]:
from tensorflow.keras.applications import MobileNetV2

# Funkcja do budowania modelu z MobileNetV2 i warstwą ArcFace
def build_model_with_arcface_loss(input_shape=(128, 128, 3), n_classes=2):
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
    # Zamrożenie warstw modelu bazowego
    base_model.trainable = False

    inputs = Input(shape=input_shape)
    labels = Input(shape=(n_classes,))
    
    x = base_model(inputs)
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(128)(x)  # Warstwa embeddingu
    
    # Dodanie warstwy ArcFaceLossLayer
    arcface_loss = ArcFaceLossLayer(n_classes=n_classes)(x, labels)
    
    model = Model(inputs=[inputs, labels], outputs=arcface_loss)
    return model

model = build_model_with_arcface_loss(input_shape=(128, 128, 3), n_classes=2)


# Kompilacja modelu z niestandardową funkcją straty i optymalizatorem
model.compile(optimizer=Adam(lr=0.0001), loss=[tf.nn.softmax_cross_entropy_with_logits], metrics=['accuracy'])

model.fit(
    train_gen_wrapper,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=50,
    validation_data=validation_gen_wrapper,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/50
43/43 [==============================] - 5s 104ms/step - loss: 7.9426 - accuracy: 0.4733 - val_loss: 5.5918 - val_accuracy: 0.7188
Epoch 2/50
43/43 [==============================] - 4s 96ms/step - loss: 5.0160 - accuracy: 0.5040 - val_loss: 3.0223 - val_accuracy: 0.6940
Epoch 3/50
43/43 [==============================] - 4s 97ms/step - loss: 2.9998 - accuracy: 0.5413 - val_loss: 1.2799 - val_accuracy: 0.8097
Epoch 4/50
43/43 [==============================] - 4s 99ms/step - loss: 1.6429 - accuracy: 0.7593 - val_loss: 0.8794 - val_accuracy: 0.8433
Epoch 5/50
43/43 [==============================] - 4s 100ms/step - loss: 0.9842 - accuracy: 0.8361 - val_loss: 0.6004 - val_accuracy: 0.9030
Epoch 6/50
43/43 [==============================] - 4s 100ms/step - loss: 0.7500 - accuracy: 0.8830 - val_loss: 0.4214 - val_accuracy: 0.9403
Epoch 7/50
43/43 [==============================] - 4s 102ms/step - loss: 0.5701 - accuracy: 0.9166 - val_loss: 0.4276 - val_accuracy: 0.9291
Epoch 8/5

In [14]:
test_loss, test_accuracy = model.evaluate(test_gen_wrapper, steps=test_generator.samples // test_generator.batch_size)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

9/9 [==============================] - 1s 86ms/step - loss: 0.3223 - accuracy: 0.9665
Test Loss: 0.3222656846046448, Test Accuracy: 0.9665427803993225
